# Predicting heart disease using machine learning

This notebook looks into using various Python-based maching learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not comeone has heart disease based on medical attributes

Approach: 1. Problem definition 2. Data 3. Evaluation 4. Features 5. Modelling 6. Experimentation

## 1. Problem Definitions
Given clinical presentations, can we determine if someone has heart disease or not?

## 2. Data
UCI Heart Disease Data Set from Kaggle

## 3. Evaluation
Goal: Try to get to 95% accuracy of predicting heart disease in patients

## 4. Features
**Data Dictionary**
* age: age in years
* sex: male or female (1 = male, 0 = female)
* cp: chest pain type (0 = typical, 1 = atypical, 2 = non-anginal, 3 = asymptomatic)
* trestbps: resting blood pressure on admission in mm Hg (above 130 is a problem)
* chol: serum cholesterol in mg/dL (above 200 is a problem)
* fbs: fasting blood sugar greater than 12 mg/dL (1 = true, 0 = false)
* restecg: resting electrocardiographic results (0 = nothing, 1 = ST-T Wave abnormality, 2 = left ventricular hypertrophy)
* thalach: max heart rate in beats per second
* exang: exercise induced angina (1 = yes, 0 = no)
* oldpeak: ST depression induced by exercise relative to rest
* slope: the slope of the peak exercise ST segment (0 = upsloping, 1 = flatsloping, 2 = downsloping)
* ca: number of major vessels (0-3) colored by fluoroscopy. The more the better
* thal: thallium stress result (3 = normal, 6 = fixed defect, 7 = reversible defect)
* target: (1 = has heart disease, 0 = does not have heart disease)

In [5]:
# Exploratory Data Analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# We want our plots to appear in the notebook
%matplotlib inline

# ML Models from sckikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# ML Model evaluation
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import confusion_matrix, classification_report, precision_score, recall_score, f1_score, plot_roc_curve

ModuleNotFoundError: No module named 'pandas'